In [1]:
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.units import units
from netCDF4 import num2date
import numpy as np
import scipy.ndimage as ndimage
from siphon.catalog import TDSCatalog
from siphon.ncss import NCSS
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import xarray as xr
from pathlib import Path
from requests.exceptions import HTTPError
from xarray.backends import NetCDF4DataStore

In [2]:
year = 2020
month = 1
day = 1
hour = 0

dt = datetime(year, month, day, hour)
dt2 = datetime(year, month, day+1, hour)

In [3]:
# Read NARR Data from THREDDS server
base_url = 'https://www.ncei.noaa.gov/thredds/catalog/model-gfs-g4-anl-files'

# Programmatically generate the URL to the day of data we want
cat = TDSCatalog(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/catalog.xml')
hours = [6,12,18,24]
precip_files = []
for hour in hours:
    time1 = dt + timedelta(hours=hour-1)
    time2 = dt + timedelta(hours=hour+1)
    precip_files.append(cat.datasets.filter_time_range(time1,time2))
    print(time1, time2, hour, precip_files)
    # ncss = precip_files[-1].subset()
    # query = ncss.query().lonlat_box(north=54, south=20, east=-65, west=-126)
    # query.all_times().variables('Total_precipitation_surface_6_Hour_Accumulation').add_lonlat() .accept('netcdf')
    # print(ncss)
print(precip_files)

2020-01-01 05:00:00 2020-01-01 07:00:00 6 [[gfsanl_4_20200101_0600_006.grb2, gfsanl_4_20200101_0600_003.grb2, gfsanl_4_20200101_0600_000.grb2]]
2020-01-01 11:00:00 2020-01-01 13:00:00 12 [[gfsanl_4_20200101_0600_006.grb2, gfsanl_4_20200101_0600_003.grb2, gfsanl_4_20200101_0600_000.grb2], [gfsanl_4_20200101_1200_006.grb2, gfsanl_4_20200101_1200_003.grb2, gfsanl_4_20200101_1200_000.grb2]]
2020-01-01 17:00:00 2020-01-01 19:00:00 18 [[gfsanl_4_20200101_0600_006.grb2, gfsanl_4_20200101_0600_003.grb2, gfsanl_4_20200101_0600_000.grb2], [gfsanl_4_20200101_1200_006.grb2, gfsanl_4_20200101_1200_003.grb2, gfsanl_4_20200101_1200_000.grb2], [gfsanl_4_20200101_1800_006.grb2, gfsanl_4_20200101_1800_003.grb2, gfsanl_4_20200101_1800_000.grb2]]
2020-01-01 23:00:00 2020-01-02 01:00:00 24 [[gfsanl_4_20200101_0600_006.grb2, gfsanl_4_20200101_0600_003.grb2, gfsanl_4_20200101_0600_000.grb2], [gfsanl_4_20200101_1200_006.grb2, gfsanl_4_20200101_1200_003.grb2, gfsanl_4_20200101_1200_000.grb2], [gfsanl_4_2020010

# Read 2 days of downloaded precip data

In [4]:

base_url = 'https://www.ncei.noaa.gov/thredds/catalog/model-gfs-g4-anl-files'

# Programmatically generate the URL to the day of data we want
cat = TDSCatalog(f'{base_url}{dt:%Y%m}/{dt:%Y%m%d}/catalog.xml')
ds = cat.datasets.filter_time_range(dt, dt2)
print(ds)
print(dt, dt2)
for file in ds:
    ncss = None
    # print(file)
    try:
        ncss = file.subset()
        query = ncss.query().lonlat_box(north=54, south=20, east=-65, west=-126)
        query.variables('Precipitation_rate_surface_6_Hour_Average').add_lonlat().accept('netcdf')
        data = ncss.get_data(query)
        print(file, flush=True)
    except HTTPError as http_err:
        print(f'HTTP error occured: {file}', flush=True)

[gfsanl_4_20200101_1800_006.grb2, gfsanl_4_20200101_1800_003.grb2, gfsanl_4_20200101_1800_000.grb2, gfsanl_4_20200101_1200_006.grb2, gfsanl_4_20200101_1200_003.grb2, gfsanl_4_20200101_1200_000.grb2, gfsanl_4_20200101_0600_006.grb2, gfsanl_4_20200101_0600_003.grb2, gfsanl_4_20200101_0600_000.grb2, gfsanl_4_20200101_0000_006.grb2, gfsanl_4_20200101_0000_003.grb2, gfsanl_4_20200101_0000_000.grb2]
2020-01-01 00:00:00 2020-01-02 00:00:00
gfsanl_4_20200101_1800_006.grb2
HTTP error occured: gfsanl_4_20200101_1800_003.grb2
HTTP error occured: gfsanl_4_20200101_1800_000.grb2
gfsanl_4_20200101_1200_006.grb2
HTTP error occured: gfsanl_4_20200101_1200_003.grb2
HTTP error occured: gfsanl_4_20200101_1200_000.grb2
gfsanl_4_20200101_0600_006.grb2
HTTP error occured: gfsanl_4_20200101_0600_003.grb2
HTTP error occured: gfsanl_4_20200101_0600_000.grb2
gfsanl_4_20200101_0000_006.grb2
HTTP error occured: gfsanl_4_20200101_0000_003.grb2
HTTP error occured: gfsanl_4_20200101_0000_000.grb2


In [4]:
for dataset in ds:
    print(type(dataset), print(dataset))

gfsanl_4_20200101_1800_006.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_1800_003.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_1800_000.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_1200_006.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_1200_003.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_1200_000.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0600_006.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0600_003.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0600_000.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0000_006.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0000_003.grb2
<class 'siphon.catalog.Dataset'> None
gfsanl_4_20200101_0000_000.grb2
<class 'siphon.catalog.Dataset'> None
